In [1]:
from searchtweets import ResultStream, gen_rule_payload, load_credentials, convert_utc_time

In [2]:
premium_search_args = load_credentials(filename="~/.twitter_keys.yaml", 
                                       yaml_key="search_tweets_fullarchive_dev",
                                       env_overwrite=True)

Grabbing bearer token from OAUTH


In [3]:
from datetime import timedelta, datetime as dt

In [125]:
fmt = '%Y-%m-%d %H:%M'
original_dt = dt.strptime("2018-02-21 21:22", fmt)
print('origin in EST:', original_dt)
minutes_to_sample = list()
curr_time = original_dt + timedelta(hours=5) # add 5 hours to set to UTC time
print('origin in UTC:', curr_time)
for i in range(1,50):
    # increment origin by one minute for start time
    delta_start = curr_time + timedelta(minutes=2) 
    delta_start_str = delta_start.strftime(fmt)
    
    # increment start time by three minutes for (exclusive) end time
    delta_end = delta_start + timedelta(minutes=2) 
    delta_end_str = delta_end.strftime(fmt)
    
    # append time range tuple to list of minutes to sample
    minutes_to_sample.append((delta_start_str, delta_end_str))
    
    curr_time = delta_start
minutes_to_sample = [(convert_utc_time(start),convert_utc_time(end)) for start,end in minutes_to_sample]
print(len(minutes_to_sample))

origin in EST: 2018-02-21 21:22:00
origin in UTC: 2018-02-22 02:22:00
49


In [126]:
print(minutes_to_sample[:5])
print(minutes_to_sample[-5:])

[('201802220224', '201802220226'), ('201802220226', '201802220228'), ('201802220228', '201802220230'), ('201802220230', '201802220232'), ('201802220232', '201802220234')]
[('201802220352', '201802220354'), ('201802220354', '201802220356'), ('201802220356', '201802220358'), ('201802220358', '201802220400'), ('201802220400', '201802220402')]


In [88]:
def query_and_save(q_rule, from_date, save_to='data',
                   premium_search_args=premium_search_args):
    import json
    import os
    from searchtweets import ResultStream
    save_path = os.path.join(save_to,'{0}.json'.format('_'.join([from_date] + q_rule.split())))
    rs = ResultStream(rule_payload=q_rule,
                  max_results=500,
                  max_pages=1,
                  **premium_search_args)
    ###return rs
    try:
        tweets = list(rs.stream())
    except Exception as e:
        print(e)
        return e
    with open(save_path, 'w') as f:
        f.write(json.dumps(tweets, indent=2))
    return tweets

In [127]:
already_queried = list()
def safe_pop(from_list, to_list):
    val = from_list.pop(0)
    to_list.append(val)
    return val

# Prototype queries

In [113]:
# first, test query 

#datetime range
dt_range = safe_pop(minutes_to_sample, already_queried)
print(dt_range)

('201802220210', '201802220212')


In [114]:
# creat test query's rule
rule = gen_rule_payload(pt_rule="#CNNTownHall", results_per_call=500, 
                        from_date=dt_range[0], to_date=dt_range[1])
print('query_rule:',rule)

query_rule: {"query": "#CNNTownHall", "maxResults": 500, "toDate": "201802220212", "fromDate": "201802220210"}


In [115]:
# make the test query
result = query_and_save(q_rule=rule, from_date=dt_range[0])

In [116]:
len(result)

500

In [118]:
len(minutes_to_sample)

90

# Iterative queries

In [132]:
for i in range(42):
    if not minutes_to_sample:
        break
    dt_range = safe_pop(minutes_to_sample, already_queried)
    print(dt_range)
    rule = gen_rule_payload(pt_rule="#CNNTownHall", results_per_call=500, 
                            from_date=dt_range[0], to_date=dt_range[1])
    print('query_rule:',rule)
    # make the test query
    result = query_and_save(q_rule=rule, from_date=dt_range[0])
    print('\t#results:',len(result))

('201802220238', '201802220240')
query_rule: {"query": "#CNNTownHall", "maxResults": 500, "toDate": "201802220240", "fromDate": "201802220238"}
	#results: 500
('201802220240', '201802220242')
query_rule: {"query": "#CNNTownHall", "maxResults": 500, "toDate": "201802220242", "fromDate": "201802220240"}
	#results: 500
('201802220242', '201802220244')
query_rule: {"query": "#CNNTownHall", "maxResults": 500, "toDate": "201802220244", "fromDate": "201802220242"}
	#results: 500
('201802220244', '201802220246')
query_rule: {"query": "#CNNTownHall", "maxResults": 500, "toDate": "201802220246", "fromDate": "201802220244"}
	#results: 500
('201802220246', '201802220248')
query_rule: {"query": "#CNNTownHall", "maxResults": 500, "toDate": "201802220248", "fromDate": "201802220246"}
	#results: 500
('201802220248', '201802220250')
query_rule: {"query": "#CNNTownHall", "maxResults": 500, "toDate": "201802220250", "fromDate": "201802220248"}
	#results: 500
('201802220250', '201802220252')
query_rule: {

In [131]:
len(minutes_to_sample)

42

In [133]:
500*66

33000